In [2]:
# Import necessary libraries

!pip install playwright
!playwright install --with-deps

import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
!pip install playwright
!playwright install --with-deps



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 19.5 MB/s eta 0:00:00
Installing dependencies...
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,309 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,653 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,230 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hi

In [3]:
import asyncio
from playwright.async_api import async_playwright
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

async def run_playwright():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Headless mode
        page = await browser.new_page()
        await page.goto("https://www.google.com")
        print(await page.title())  # Should print "Google"
        await browser.close()

# Run the asynchronous function using asyncio.run
asyncio.run(run_playwright())

Google


# Check Captcha

In [4]:
import asyncio
from playwright.async_api import async_playwright

async def check_captcha(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto(url, timeout=60000)
            await page.wait_for_load_state("load")
            captcha_text = await page.query_selector("div:has-text('CAPTCHA')")
            captcha_frame = await page.query_selector("iframe[src*='captcha']")
            captcha_image = await page.query_selector("img[src*='captcha']")
            if captcha_text or captcha_frame or captcha_image:
                print("CAPTCHA detected on the site.")
            else:
                print("No CAPTCHA detected on the site.")
        except Exception as e:
            print(f"Error accessing the site: {e}")
        finally:
            await browser.close()

# Extract data


In [5]:
import asyncio
from playwright.async_api import async_playwright
import pandas as pd
import nest_asyncio
import re

nest_asyncio.apply()

def extract_numbers(text):
    """Extracts numbers or ranges of numbers from a string."""
    ranges = re.findall(r'\d+\.?\d*', text.replace(',', ''))
    return ' - '.join(ranges)

async def scrape_immobiliare(base_url, max_pages=1):  # Set max_pages to 1 for testing
    data = []

    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()

        # Scrape multiple pages
        for page_num in range(1, max_pages + 1):
            url_with_page = f"{base_url}?pag={page_num}"
            print(f"Scraping {url_with_page}...")

            # Go to the target URL
            await page.goto(url_with_page)

            # Wait for the listings to load
            try:
                await page.wait_for_selector(".in-listingCardPropertyContent, .re-overview__price.has-range", timeout=90000)
            except Exception as e:
                print(f"Timeout error: {e}")
                content = await page.content()
                with open(f"page_content_{page_num}.html", "w") as file:
                    file.write(content)
                await page.screenshot(path=f"timeout_screenshot_{page_num}.png")
                break

            # Extract data
            listings = await page.query_selector_all(".in-listingCardPropertyContent")
            for listing in listings:
                try:
                    title_element = await listing.query_selector(".in-listingCardTitle")
                    price_element = await listing.query_selector(".in-formattedPrice__text, .in-listingCardPrice span")
                    link_element = await listing.query_selector(".in-listingCardTitle")
                    range_price_element = await listing.query_selector(".re-overview__price.has-range span")

                    inserzionista_element = await listing.query_selector(".in-listingCardAgencyLogo__image")
                    phone_element = await listing.query_selector(".in-agencyPhone")

                    title = await title_element.inner_text() if title_element else 'N/A'
                    link = await link_element.get_attribute("href") if link_element else 'N/A'
                    price = None
                    range_price = None
                    inserzionista = await inserzionista_element.get_attribute("alt") if inserzionista_element else 'N/A'
                    phone = await phone_element.get_attribute("href") if phone_element else 'N/A'
                    phone = phone.replace("tel:", "").strip() if phone != 'N/A' else phone

                    # Check if there is a range price
                    if range_price_element:
                        range_price_text = await range_price_element.inner_text()
                        range_price = range_price_text.strip()
                        price = 0  # Set Price to 0 if range price is present
                        print(f"Found price range: {range_price_text} -> {range_price}")
                    else:
                        price_text = await price_element.inner_text() if price_element else ''
                        price = re.sub(r'[^\d]', '', price_text) if price_text else None
                        if price:
                            price = float(price)
                        print(f"Found price: {price_text} -> {price}")

                    full_link = link

                    # Extract additional features if available
                    features = await listing.query_selector_all(".in-listingCardFeatureList__item, .re-mainFeatures__item.flex")
                    locali = ""
                    superficie = ""
                    tipologie = ""
                    bagni = ""
                    piano = ""
                    ascensore = ""
                    balcone = ""

                    if features:
                        for feature in features:
                            text = await feature.inner_text()
                            if "locali" in text or re.search(r'\b-\b', text):
                                locali = extract_numbers(text)
                            elif "m²" in text or "da" in text:
                                # Extract only numbers from superficie, keep ranges as they are
                                superficie = extract_numbers(text)
                            elif "tipologie" in text:
                                tipologie = extract_numbers(text)
                            elif "bagno" in text:
                                bagni = extract_numbers(text)
                            elif "Piano" in text:
                                piano = extract_numbers(text)
                            elif "Ascensore" in text:
                                ascensore = "Yes"
                            elif "Balcone" in text:
                                balcone = "Yes"

                    data.append({
                        "Title": title.strip(),
                        "Price": price if price is not None else "",
                        "Price Range": range_price if range_price else "",
                        "Link": full_link,
                        "Inserzionista": inserzionista.strip(),
                        "Phone": phone,
                        "Locali": locali.strip(),
                        "Superficie, m²": superficie.strip(),
                        "Tipologie": tipologie.strip(),
                        "Bagni": bagni.strip(),
                        "Piano": piano.strip(),
                        "Ascensore": ascensore.strip(),
                        "Balcone": balcone.strip()
                    })
                except Exception as e:
                    print(f"Error scraping listing: {e}")

            # Go to the next page if available
            next_button = await page.query_selector("a[aria-label='Next']")
            if next_button:
                await next_button.click()
                await page.wait_for_load_state("load")
            else:
                break

        await browser.close()

    # Save to DataFrame
    return pd.DataFrame(data)

# URL for Milan apartments
base_url = "https://www.immobiliare.it/vendita-appartamenti/milano/"

# Run the async function
df = asyncio.run(scrape_immobiliare(base_url, max_pages=1))  # Set max_pages to 1 for testing

# Save to CSV
df.to_csv("immobiliare_listings.csv", index=False)
print(df.head())

Scraping https://www.immobiliare.it/vendita-appartamenti/milano/?pag=1...
Found price: da € 386.000 -> 386000.0
Found price: da € 369.500 -> 369500.0
Found price: da € 244.000 -> 244000.0
Found price: € 735.000 -> 735000.0
Found price: € 445.000 -> 445000.0
Found price: € 159.000 -> 159000.0
Found price: € 252.000 -> 252000.0
Found price: € 450.000 -> 450000.0
Found price: € 280.000 -> 280000.0
Found price: € 725.000 -> 725000.0
Found price: € 420.000 -> 420000.0
Found price: € 327.000 -> 327000.0
Found price: € 465.000 -> 465000.0
Found price: da € 192.000 -> 192000.0
Found price: € 790.000 -> 790000.0
Found price: € 1.420.000 -> 1420000.0
Found price: € 1.300.000 -> 1300000.0
Found price: € 1.090.000 -> 1090000.0
Found price: € 405.000 -> 405000.0
Found price: € 295.000 -> 295000.0
Found price: € 285.000 -> 285000.0
Found price: € 1.450.000 -> 1450000.0
Found price: € 430.000 -> 430000.0
Found price: € 395.000 -> 395000.0
Found price: € 1.610.000 -> 1610000.0
                        

# create report

In [6]:
!pip install ace_tools

In [10]:
from openpyxl import Workbook
from openpyxl.utils import FORMULAE

# Load the data from the file
file_path = "immobiliare_listings-bigdata.csv"
data = pd.read_csv("/content/immobiliare_listings-bigdata.csv", encoding="latin-1")

# Filter properties with "Superficie, m²" > 200
filtered_data = data[data["Superficie, m²"] > 200]

# Fixed parameters for calculations
bilocale_price = 350000  # EUR
cost_design = 8000  # EUR
cost_notary = 5000  # EUR
agent_commission_rate = 0.04  # 4%
renovation_works_cost = 90000  # EUR
materials_cost = 80000  # EUR
unforeseen_costs = 10000  # EUR
consulting_fees = 8000  # EUR
imu_tari_admin = 2000  # EUR
furniture_cost = 20000  # EUR
utilities_cost = 2000  # EUR
counters_cost = 2000  # EUR
# Create an updated Excel workbook
wb_updated = Workbook()

for index, row in filtered_data.iterrows():
    ws = wb_updated.create_sheet(title=f"Property {index + 1}")


    # Revenue Section
    ws.append(["REVENUE"])
    ws.append(["Immobile 1 - Bilocale 48m²", f"=350000"])
    ws.append(["Immobile 2 - Bilocale 48m²", f"=350000"])
    ws.append(["Immobile 3 - Bilocale 48m²", f"=350000"])
    ws.append(["Immobile 4 - Bilocale 48m²", f"=350000"])
    ws.append(["TOTAL REVENUE", "=SUM(B2:B5)"])

    ws.append([])
    ws.append(["COSTO INTERVENTO"])
    ws.append(["Costo immobile da acquistare", row["Prezzo, EUR"], None, row["Titolo"], f"{row['Superficie, m²']} m²", row["Link"]])
    ws.append(["Imposta di registro", f"=B9*0.09"])
    ws.append(["Notaio", 5000])
    ws.append(["Progettazione", 8000])
    ws.append(["Proviggione agente immobiliare", f"=B9*0.04"])
    ws.append(["IVA P. AI.", f"=B13*0.22"])
    ws.append(["Imprevisti", unforeseen_costs])
    ws.append(["Costo opere murarie, idrauliche, elettriche", renovation_works_cost])
    ws.append(["Costo materiali e mobili bagni, rivestimenti e pavimenti", materials_cost])
    ws.append(["IVA Ristrutturazione", f"=(B16+B17)*0.22"])
    ws.append(["TOTAL CoGS", "=SUM(B9:B18)"])

    ws.append([])
    ws.append(["Spese"])
    ws.append(["Consulting fees", consulting_fees])
    ws.append(["IMU / TARI / Spese di amministrazione", imu_tari_admin])
    ws.append(["Mobilio", furniture_cost])
    ws.append(["Utenze", utilities_cost])
    ws.append(["Contatori / Citofoni / Cassette Postali", counters_cost])
    ws.append(["Totale spese", "=SUM(B22:B26)"])
    ws.append(["Gran totale spese operazione", "=B19+B27"])

    ws.append([])
    ws.append(["Altre spese (Se ci sono)"])
    ws.append(["Da inserire", 0.0])
    ws.append(["Da inserire", 0.0])
    ws.append(["TOTAL OTHER EXPENSES", "=SUM(B31:B32)"])

    ws.append([])
    ws.append(["NET PROFIT/LOSS, EUR", "=B6-B28-B33"])
    ws.append(["NET PROFIT/LOSS %", "=(B35/B6)*100"])



# Save the updated workbook
output_updated_path = "conto_economico_updated_formulas.xlsx"
wb_updated.save(output_updated_path)

print(f"Updated workbook with formulas saved as '{output_updated_path}'.")


Updated workbook with formulas saved as 'conto_economico_updated_formulas.xlsx'.
